## Notebook for creating efavor files for running the SWDE case study

### T. Janus, 28-11-2024

In [ ]:
# Library import
import pathlib
import pandas as pd
from pyfavor.utils import hr_to_sec, hr_to_min, read_table, find_os, install_package
from pyfavor.create import (
    create_times_df, create_inlets_df, create_flows_df, create_pressures_df, 
    get_logger_info, get_notes, write_efavor_pressures, read_pressure_setpoints)
# Input file names
pressure_measurement_file = pathlib.Path(
    "efavor/swde_case_study/bul_data/pressure_measurements.xlsx")
# Output file names
logger_file = pathlib.Path("efavor/swde_case_study/outputs/logger_file.xlsx")
pressure_measurement_file_filled = pathlib.Path(
    "efavor/swde_case_study/outputs/pressure_measurements_filled.xlsx")
# Constants and global variables
inlet_name: str = "A"

In [ ]:
## Missing value imputer for pressure dataset
# Create a report for imputed values
imputed_report = []

# Function to impute missing pressure values in the pressure measurement dataframe
def impute_values(group):
    missing = group[group["Pressure"].isna()]
    for idx in missing.index:
        imputed_value = group.loc[:idx, "Pressure"].mean()
        imputed_report.append(
            {"Time": idx[0], "node": idx[1], "Imputed_Pressure": imputed_value})
        group.at[idx, "Pressure"] = imputed_value
    return group

# Define the function for the pipeline
def convert_pressure_to_metres_h2o(df):
    df["Pressure"] = df["Pressure"] * 10.1972
    return df

In [ ]:
# 1a.Read logger data from pressure data file
inlet_nodes = ["J465"]
start_logger_id: int = 1
# Find unique nodes from the pressure measurement file
nodes_list = list(pd.read_excel(pressure_measurement_file).\
    loc[:,'_node'].unique())
print(f"Found {len(nodes_list)} unique nodes in the pressure data file.")

In [ ]:
# 1b Create the logger data file
column_values = {
    "Logger ID": None, 
    "Nearest Node ID (EPANET Junction ID)": nodes_list, 
    "Logger Elevation (use -1000 if the same as node elevation) [m]": -1_000,
    "Is inlet node? (true/false)": "False"
}
df_logger_dict = {}
for col_name, values in column_values.items():
    if col_name == "Logger ID":
        df_logger_dict[col_name] = list(range(start_logger_id, len(nodes_list)+start_logger_id))
    elif isinstance(values, list):
        df_logger_dict[col_name] = values
    else:
        df_logger_dict[col_name] = [values for _ in range(0,len(nodes_list))]

df_logger = pd.DataFrame.from_dict(df_logger_dict, orient='columns').set_index("Logger ID")
df_logger.loc[1,"Is inlet node? (true/false)"] = "True"
notes_data = [["Logger neighbourhood parameter = 0"], 
              ["Head difference tolerance parameter [m] = 0.01"]]
notes_df = pd.DataFrame(notes_data)
with pd.ExcelWriter(logger_file, engine="openpyxl") as writer:
    df_logger.to_excel(writer, sheet_name="loggers", index=True)
    notes_df.to_excel(writer, sheet_name="notes", header=False, index=False)

In [ ]:
# Build the efavor input file
logger_info = get_logger_info(logger_file)
notes = get_notes(logger_file)
junction_ids = logger_info['Nearest Node ID (EPANET Junction ID)'].to_list()
inlet_nodes = logger_info[logger_info["Is inlet node? (true/false)"] == True]
inlet_logger_ids = inlet_nodes['Logger ID'].to_list()
inlet_logger_junction_ids = inlet_nodes['Nearest Node ID (EPANET Junction ID)'].to_list()
# Create a mapping between logger IDs and EPANET Junction IDS
epanet_to_logger_id_map = \
    dict(zip(logger_info['Nearest Node ID (EPANET Junction ID)'], logger_info['Logger ID']))

## Fill missing values in the pressure spreadsheet and save to a new file

In [ ]:
# Function to fill in missing rows
def fill_missing_values(df):
    # Create an empty list to store new rows
    new_rows = []
    # Iterate over each node group
    for node, group in df.groupby('_node'):
        # Create a time range from 01:05:00 to 04:00:00 with 5-minute intervals
        full_time_range = pd.date_range(
            start="2024-11-20 01:05:00", 
            end="2024-11-20 04:00:00", 
            freq='5T')
        # Reindex the group to include all times in the full time range
        group = group.set_index('Time').reindex(
            full_time_range, method=None).reset_index()
        group.rename(columns={'index': 'Time'}, inplace=True)
        # Iterate over the rows to fill missing data
        for i in range(1, len(group)):
            # Only fill missing rows (NaN values)
            if pd.isna(group.loc[i, "Pressure"]):
                # Get the previous and next values for Pressure
                prev_value = group.loc[i - 1, "Pressure"]
                next_value = group.loc[i + 1, "Pressure"] if i + 1 < len(group) else prev_value
                # Handle edge cases (01:05:00 and 04:00:00)
                if group.loc[i, "Time"] == pd.Timestamp("2024-11-20 01:05:00"):
                    group.at[i, "Pressure"] = group.loc[i + 1, "Pressure"]  # Use 01:10:00 value
                    group.at[i, "_node"] = group.loc[i + 1, "_node"]  # Use downstream node
                elif group.loc[i, "Time"] == pd.Timestamp("2024-11-20 04:00:00"):
                    group.at[i, "Pressure"] = group.loc[i - 1, "Pressure"]  # Use 03:55:00 value
                    group.at[i, "_node"] = group.loc[i - 1, "_node"]  # Use upstream node
                else:
                    group.at[i, "Pressure"] = (prev_value + next_value) / 2  # Mean of previous and next
                    group.at[i, "_node"] = group.loc[i + 1, "_node"]  # Use downstream node
        # Append the filled group to the new rows list
        new_rows.append(group)
    # Concatenate all the filled groups back together
    return pd.concat(new_rows)

pressure_measurements = pd.read_excel(pressure_measurement_file)
pressure_measurements = pressure_measurements.sort_values(by=["_node", "Time"])
pressure_measurements_filled = fill_missing_values(pressure_measurements)
pressure_measurements_filled.to_excel(pressure_measurement_file_filled, index=False)

## Create individual sheets

In [ ]:
# Create pressures dataframe
swde_data = pd.read_excel(pressure_measurement_file_filled)\
    .pipe(convert_pressure_to_metres_h2o)\
    .rename(columns={"_node": "node"})\
    .set_index("Time")\
    .loc[lambda x: ~x.index.isna()]\
    .loc[lambda x: x.index.time >= pd.to_datetime("01:05:00").time()]\
    .groupby("node")\
    .resample('15T', label='right', closed='right')\
    .mean()\
    .reset_index()\
    .pivot(index='Time', columns='node', values='Pressure')

swde_data_efavor = swde_data\
    .assign(Inlet=lambda x: "A")\
    .set_index("Inlet")\
    .rename(columns=epanet_to_logger_id_map)

In [ ]:
swde_data_efavor = swde_data_efavor[sorted(swde_data_efavor.columns, key=lambda x: int(x))]
swde_data_efavor.head(100)